<font color="#CA3532"><h1 align="left">**Aprendizaje por Refuerzo**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Multiarmed Bandits Contextuales**</h2></font>

# **Multiarmed bandits para Credit Scoring**

El objetivo es construir un sistema automático de aprendizaje por refuerzo para gestionar un sistema de concesión de créditos.
La idea es que cuando un cliente contacte con el banco para solicitar un crédito, el sistema se lo conceda (acción 1) o no (acción 0).
Por tanto el sistema de aprendizaje por refuerzo tiene **dos posibles acciones**.

Por otra parte, el reward obtenido es:

- 0 si el sistema decide no ofrecerle el crédito al cliente
- 1 si se le ofrece el crédito y **lo paga**
- -10 si se le ofrece crédito y **no lo paga**

Para construir el sistema disponemos del siguiente dataset: https://drive.google.com/file/d/1TdTDAjndt5sn_7HKwc1PifcaQTRNHzkR/view?usp=sharing


A continuación se descarga y muestra:

In [ ]:
COLAB = True

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 200

# Descargamos dataset:
aux = "'https://docs.google.com/uc?export=download&id=1000WA5q8DTZ68uoPaXddJ1TLJPBsIZpr&confirm=t'"
!wget -q $aux -O ./cs.csv
aux = "'https://docs.google.com/uc?export=download&id=11SdrmYpGwW7xC6Nooe6aRBkORBnirrkm&confirm=t'"
!wget -q $aux -O ./DataDictionary.csv

**Información sobre cada variable:**

In [ ]:
info_variables = pd.read_csv("DataDictionary.csv", sep=";").set_index("Variable Name")
info_variables

**Base de datos:**

In [ ]:
df = pd.read_csv("./cs.csv")
df.head(10)

In [ ]:
df.describe().T[["count", "min", "max", "mean", "std"]]

In [ ]:
df.isna().sum()

In [ ]:
(df["MonthlyIncome"].isna() & df["NumberOfDependents"].isna()).sum()

In [ ]:
df["age"].hist(bins=20);

In [ ]:
df["NumberOfTime30-59DaysPastDueNotWorse"].value_counts()

In [ ]:
# limpieza básica:
#
# 1- Eliminamos filas con missing values (MonthlyIncome y NumberOfDependents)
# 2- Nos quedamos solo con personas de 18 a 70 años (incluidos)
# 3- Eliminamos filas que tengan 96 o 98 en columnas de retraso
#    (NumberOfTime30-59DaysPastDueNotWorse, NumberOfTime60-89DaysPastDueNotWorse,
#     NumberOfTimes90DaysLate)

df.dropna(inplace=True)
df = df[ (df["age"]>=18) & (df["age"]<=70) ]

df = df[ ~df["NumberOfTime30-59DaysPastDueNotWorse"].isin([96, 98]) ]
df = df[ ~df["NumberOfTime60-89DaysPastDueNotWorse"].isin([96, 98]) ]
df = df[ ~df["NumberOfTimes90DaysLate"].isin([96, 98]) ]

In [ ]:
df.describe().T

La columna **SeriousDlqin2yrs** es 0 si el cliente paga el crédito, y 1 si no lo paga.

Ahora dividimos el dataset en training y test:

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, random_state=1, test_size=0.3, stratify=df["SeriousDlqin2yrs"])

In [ ]:
# frecuencia de impagos en training y test (en tanto por 1):
df_train["SeriousDlqin2yrs"].mean(), df_test["SeriousDlqin2yrs"].mean()

In [ ]:
# Defino clase clientes:

class clientes:
    def __init__(self, df): # self: parámetros internos
        # mi clase va a tener dos parámetros internos:
        # i, df
        self.i = -1 # i: contador (de clientes); índice al último cliente mostrado
        self.df = df
    def cliente(self): # va a simular que llega un cliente
        self.i = self.i + 1
        if self.i == len(self.df):
            self.i = 0
    def get_context(self): # va a extraer datos de ese cliente
        return self.df.iloc[self.i].drop("SeriousDlqin2yrs")
    def get_reward(self, accion): # reward con ese cliente
        # calculo reward:
        if accion==0: # no se da crédito
            return 0
        # se da crédito:
        if self.df.iloc[self.i]["SeriousDlqin2yrs"] == 1: # no paga
            return -10
        return 1 # paga

In [ ]:
# clientes: una clase
# cl_train: una instancia de esa clase
# cl_test: otra instancia de esa clase
#
# podemos ver cl_train como una "caja" donde hemos
# metido el dataset de entrenamiento y va a haber
# métodos que me permiten interaccionar con él
#
# Lo mismo con cl_test, pero para los datos de test

cl_train = clientes(df_train)
cl_test  = clientes(df_test)

In [ ]:
cl_train.cliente() # simulo que llega un cliente nuevo (eligir una fila alea del dataset)

In [ ]:
cl_train.get_context() # obtengo datos de ese cliente

In [ ]:
cl_train.get_context().values

In [ ]:
cl_train.cliente()
cl_train.get_context()

In [ ]:
cl_train.get_reward(1) # se da crédito y paga

# **Tareas a realizar**

1- Calcula la evolución del reward total, regret total de un algoritmo aleatorio en training. Calcula en otra gráfica la evolución de su reward promedio y regret promedio.

2- Entrena un multiarmed bandit contextual lineal en training. Calcula las mismas métricas que en el punto 1

3- Calcula, para algunos clientes de training, las Q dadas por el modelo para cada acción. Chequea si obtienes los mismos valores calculándolos tú. Chequea si la acción elegida por el modelo es la de mayor Q.

4- Calcula en test el rendimiento del algoritmo de multiarmed bandit. Compáralo con el grupo de control de test (reserva un 10% de clientes en test como grupo de control).

5- Obtén la interpretabilidad del modelo. ¿En qué variables se está fijando para decidir?

6- (Opcional). Trata de optimizar tu sistema haciendo selección de variables.

### Tarea 1

In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
N = 10000

historico_reward_promedio = []
historico_reward_total    = []
historico_regret_total    = []
reward_total = 0
regret_total = 0
for i in tqdm(range(N)):
  cl_train.cliente() # cliente con el que contacto
  accion = np.random.choice(2)
  reward = cl_train.get_reward(accion)

  reward_total += reward
  ideal = max(cl_train.get_reward(0), cl_train.get_reward(1))
  regret = ideal - reward
  regret_total += regret
  historico_reward_total.append(reward_total)
  historico_reward_promedio.append(reward_total/(i+1)) # i+1 es el número de iteraciones
  historico_regret_total.append(regret_total)

print("Reward promedio:", reward_total/N)
plt.plot(historico_reward_total)
plt.title("Evolución del reward total")
plt.show()
plt.plot(historico_reward_promedio)
plt.title("Evolución del reward promedio");
plt.show()
plt.plot(historico_regret_total)
plt.title("Evolución del regret total");

In [ ]:
historico_regret_total[-1] / N

### Tarea 2

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd

    gdd.download_file_from_google_drive(file_id='1fCnGzS5U_x-k_03op_XJkHVS4jpvjSxS',
                                        dest_path='./spacebandits.zip', unzip=True)

In [ ]:
from space_bandits import LinearBandits

n_features = 10
n_acciones = 2

In [ ]:
N = 10000

agente = LinearBandits(n_acciones, n_features, initial_pulls=100) # initial_pulls: duración de la fase de exploración pura

historico_reward_promedio = []
historico_reward_total    = []
historico_regret_total    = []
reward_total = 0
regret_total = 0
for i in tqdm(range(N)):
  cl_train.cliente() # cliente con el que contacto
  contexto = cl_train.get_context().values
  accion = agente.action(contexto)
  reward = cl_train.get_reward(accion)
  agente.update(contexto, accion, reward)

  reward_total += reward
  ideal = max(cl_train.get_reward(0), cl_train.get_reward(1))
  regret = ideal - reward
  regret_total += regret
  historico_reward_total.append(reward_total)
  historico_reward_promedio.append(reward_total/(i+1)) # i+1 es el número de iteraciones
  historico_regret_total.append(regret_total)

print("Reward promedio:", reward_total/N)
plt.plot(historico_reward_total)
plt.title("Evolución del reward total")
plt.show()
plt.plot(historico_reward_promedio)
plt.title("Evolución del reward promedio");
plt.show()
plt.plot(historico_regret_total)
plt.title("Evolución del regret total");

### Tarea 3

In [ ]:
cl_train.cliente()

In [ ]:
cl_train.get_context()

In [ ]:
contexto = cl_train.get_context().values
agente.action(contexto)

In [ ]:
agente.expected_values(contexto)

In [ ]:
agente.mu

In [ ]:
# acción 1:
agente.mu[1][-1] + (agente.mu[1][:-1]*contexto).sum()

In [ ]:
# acción 0:
agente.mu[0][-1] + (agente.mu[0][:-1]*contexto).sum()

### Tarea 4

### Tarea 5

In [ ]:
# Primero creo una tabla donde tengo clientes y las predicciones del modelo en ellos

df_subrogado = df_test.drop("SeriousDlqin2yrs", axis=1)
df_subrogado

In [ ]:
df_subrogado.columns

In [ ]:
cl_train.get_context()

In [ ]:
acciones_predichas = []
contextos = df_subrogado.values
for x in contextos: # me recorro fila a fila el dataset subrogado
  accion = agente.action(x)
  acciones_predichas.append(accion)

In [ ]:
len(acciones_predichas)

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from graphviz import Source

startbold = '\033[1m'
endbold = '\033[0m'

clf = DecisionTreeClassifier(
    #max_depth=1,
    #min_samples_split=0.3, # mínimo número de casos en un nodo para partirlo. 0.3 = 30% del total de casos
    #min_samples_leaf=0.2,  # mínimo número de casos en nodo hoja. 0.2 = 20% del total de casos
    max_leaf_nodes=5, # máximo número de nodos hoja
    #min_weight_fraction_leaf=0.05,
    #min_impurity_decrease=0.15
    )

clf = clf.fit(contextos, acciones_predichas)
# clf.classes_
attributes_names = df_subrogado.columns

display(Source( export_graphviz(clf, out_file=None,
                                feature_names=attributes_names,
                                #class_names=cluster_names,
                                filled=True, rounded=True,
                                special_characters=True,
                                impurity=True,
                                leaves_parallel=True,
                                rotate=False,
                                node_ids=True)))

In [ ]:
np.unique(acciones_predichas, return_counts=True)

In [ ]:
len(df_train)

In [ ]:
penalty = -10
((penalty*(df_train["SeriousDlqin2yrs"]==1)).sum() + (1*(df_train["SeriousDlqin2yrs"]==0)).sum()) / len(df_train)